In [6]:
import cv2
import numpy as np
from numpy import pi
from math import hypot
from skimage.exposure import rescale_intensity
from scipy import ndimage

In [59]:
def max_filter(image,size):
    """
    Min Filter Over the image
    """
    outputImage = np.zeros_like(image)
    l = int((size-1)/2)
    imagePadded = np.zeros((image.shape[0]+2*l,image.shape[1]+2*l))
    imagePadded[l:-l,l:-l] = image
    for i in range(image.shape[1]):
        for j in range(image.shape[0]):
            outputImage[j,i] = np.amax(imagePadded[j:j+2*l+1,i:i+2*l+1])
    return outputImage

def min_filter(image,size):
    """
    Max Filter over the image
    """
    outputImage = np.zeros_like(image)
    l = int((size-1)/2)
    imagePadded = np.zeros((image.shape[0]+2*l,image.shape[1]+2*l))
    imagePadded[l:-l,l:-l] = image
    for i in range(image.shape[1]):
        for j in range(image.shape[0]):
            outputImage[j,i] = np.amin(imagePadded[j:j+2*l+1,i:i+2*l+1])
    return outputImage


def lineDetection(image,rhoQuant,thetaQuant,threshold):
    """
    A function for detecting lines in an image
    """
    r,c = image.shape
    edgeImage = cv2.Canny(image,75,175)
    rhoMax = hypot(r,c)
    accumulator = np.zeros((rhoQuant,thetaQuant))
    for x in range(r):
        for y in range(c):
            if edgeImage[x,y] == 0: continue
            for theta in range(thetaQuant):
                thetaVal = (theta*pi/thetaQuant)
                rhoVal = (x*np.cos(thetaVal) + y*np.sin(thetaVal))
                rho = int((rhoQuant*rhoVal/rhoMax))
                accumulator[rho,theta] += 1
    houghOutput = rescale_intensity(accumulator, in_range=(0, 255))
    houghOutput = (houghOutput*255).astype("uint8")

    maxHough = max_filter(accumulator,20)
    maxBin = (maxHough == accumulator)
    minHough = min_filter(accumulator,20)
    houghDifference = ((maxHough-minHough)>threshold)
    maxBin[houghDifference == 0] = 0
    params = []
    
    for rho in range(rhoQuant):
        for theta in range(thetaQuant):
            if maxBin[rho,theta]:
                params.append((rho,(pi/2 - theta*pi/thetaQuant)))

    return params,houghOutput

#### Testing on 3 different images

In [61]:
t1Orig = cv2.imread("test.bmp")
t1BW = cv2.cvtColor(t1Orig, cv2.COLOR_BGR2GRAY)
t2Orig = cv2.imread("test2.bmp")
t2BW = cv2.cvtColor(t2Orig, cv2.COLOR_BGR2GRAY)
inOrg = cv2.imread("input.bmp")
inBW = cv2.cvtColor(inOrg, cv2.COLOR_BGR2GRAY)
parameters,houghOutput = lineDetection(t1BW,360,480,50)

for rho,theta in parameters:
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 1000*(-b))
    y1 = int(y0 + 1000*(a))
    x2 = int(x0 - 1000*(-b))
    y2 = int(y0 - 1000*(a))

    cv2.line(t1Orig,(x1,y1),(x2,y2),(0,0,255),1)
cv2.imwrite('a11.jpg', t1Orig)
cv2.imwrite('a12.jpg', houghOutput)

#We are writiing the output to files a11 and a12


cv2.waitKey(0)
cv2.destroyAllWindows()